[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/bin/bin_simple.ipynb)

# Bin - Quick Demo

Interactive calibration-aware 4D-STEM binning with BF/ADF quality checks.
Uses synthetic SrTiO3 crystal data with realistic Bragg spots.

In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import numpy as np
import torch
from quantem.widget import Bin

In [3]:
# --- Synthetic 4D-STEM: SrTiO3 crystal with Bragg spots (PyTorch GPU) ---
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

scan_rows, scan_cols = 64, 64
det_rows, det_cols = 256, 256
pixel_size = 2.39   # Angstrom/px
k_pixel_size = 0.46  # mrad/px

# Detector coordinate grid
dr = torch.arange(det_rows, device=device, dtype=torch.float32) - det_rows / 2
dc = torch.arange(det_cols, device=device, dtype=torch.float32) - det_cols / 2
KY, KX = torch.meshgrid(dr, dc, indexing='ij')
kr2 = KY**2 + KX**2

# Central beam (bright field disk)
central_beam = torch.exp(-kr2 / (2 * 32.0**2))

# Bragg spots — cubic perovskite reciprocal lattice
lattice_a = 54.0
bragg_spots = torch.zeros(det_rows, det_cols, device=device)
for h in range(-2, 3):
    for k in range(-2, 3):
        if h == 0 and k == 0:
            continue
        dist2 = (KY - h * lattice_a)**2 + (KX - k * lattice_a)**2
        intensity = 0.3 * torch.exp(torch.tensor(-0.1 * (h**2 + k**2), device=device))
        bragg_spots += intensity * torch.exp(-dist2 / (2 * 6.0**2))

base_dp = central_beam + bragg_spots

# Scan-position-dependent intensity (atom column contrast)
si = torch.linspace(-1, 1, scan_rows, device=device)
sj = torch.linspace(-1, 1, scan_cols, device=device)
SY, SX = torch.meshgrid(si, sj, indexing='ij')
atom_contrast = 1.0 + 0.15 * torch.sin(2 * torch.pi * SY * 4) * torch.sin(2 * torch.pi * SX * 4)

# Build 4D with broadcasting: (scan_rows, scan_cols, det_rows, det_cols)
data4d = base_dp.unsqueeze(0).unsqueeze(0) * atom_contrast.unsqueeze(-1).unsqueeze(-1)

# Poisson noise (NumPy — torch.poisson unreliable on MPS)
data_np = data4d.clamp(min=0).cpu().numpy() * 100
data = np.random.default_rng(42).poisson(data_np).astype(np.float32)

size_gb = data.nbytes / 1024**3
print(f'Synthetic 4D-STEM: {data.shape} ({size_gb:.2f} GB)')

Using device: mps
Synthetic 4D-STEM: (64, 64, 256, 256) (1.00 GB)


In [4]:
w = Bin(data, pixel_size=pixel_size, k_pixel_size=k_pixel_size, det_bin_row=8, det_bin_col=8)
w

Bin(shape=(64, 64, 256, 256), bin=(1, 1, 8, 8), binned_shape=(64, 64, 32, 32), mode=mean, edge=crop, device=mps)

In [5]:
# Access binned tensor for downstream analysis
binned = w.get_binned_data(copy=False)
print(type(binned), tuple(binned.shape))

<class 'torch.Tensor'> (64, 64, 32, 32)
